In [1]:
import re
import string
from nltk.tokenize import WhitespaceTokenizer 

In [2]:
white_space_tokenizer = WhitespaceTokenizer()

In [3]:
file_name_40_sentences = "sample_sentences.txt"

file_name_dictionary = "Viet74K.txt"

**Load data**

In [4]:
sentences = list()

with open(file_name_40_sentences, "r", encoding='utf-8') as f:
    content = f.readlines()
    f.close()

    for i in range(len(content)):
        sentence = content[i].strip()
        sentences.append(sentence)
        
print("Number of senteces in data:", len(sentences))

Number of senteces in data: 40


**Load dictionary: Vietnamese**

In [5]:
dictionary_corpus = list()

with open(file_name_dictionary, "r", encoding='utf-8') as f:
    content = f.readlines()
    f.close()

    for i in range(len(content)):
        term = content[i].strip().lower()
        dictionary_corpus.append(term)
        
dictionary_corpus

print("Number of terms in dictionary:", len(dictionary_corpus))

Number of terms in dictionary: 73901


**Maximum Matching**

In [6]:
def PreprocessString(string_data):
    # Lower the text
    preprocess_data = string_data.lower()
    
    # Remove punctuations, each punctuation = space, ex: ""information @#$retrieval" -> "information    retrieval"
    preprocess_data = re.sub('[%s]' % re.escape(string.punctuation), ' ', preprocess_data)   
        
    # Tokenize word by white space
    preprocess_data = white_space_tokenizer.tokenize(preprocess_data)
      
    return preprocess_data

In [14]:
sent_token = PreprocessString('tôi đi đánh cầu lông')
print(sent_token)

['tôi', 'đi', 'đánh', 'cầu', 'lông']


In [15]:
GetTerm(sent_token, 3, 5)

'cầu lông'

In [11]:
def GetTerm(sentence_tokenized, start_index, end_index):
    term = ""
    for i in range(start_index, end_index):
        term += sentence_tokenized[i] + " "
    
    return term.strip()

In [18]:
LENGTH_MAX_TERM_VIETNAMESE = 4
LENGTH_MAX_TERM = LENGTH_MAX_TERM_VIETNAMESE

sentences_tokenized = dict()

for ith_sentence, sentence in enumerate(sentences):
    """ Hiện tại thì chỗ này xét cả câu luôn, về sau sẽ nâng cấp lên thành xét các vế trong câu """
    sentence_part_tokenzied = PreprocessString(sentence)
    
    ith_start_term = 0
    if len(sentence_part_tokenzied) > LENGTH_MAX_TERM:
        ith_end_term = ith_start_term + LENGTH_MAX_TERM
    else:
        ith_end_term = len(sentence_part_tokenzied)
        
    term = GetTerm(sentence_part_tokenzied, ith_start_term, ith_end_term)
    
    # Xét hết cả vế
    while True:
        
        # Xét hết cả term
        while ith_start_term != ith_end_term:
            if term in dictionary_corpus:
                
                if ith_sentence not in sentences_tokenized.keys():
                    sentences_tokenized[ith_sentence] = [(term, ith_start_term, ith_end_term-1)]
                else:
                    sentences_tokenized[ith_sentence].append((term, ith_start_term, ith_end_term-1))
#                 print("Trùng được term:", term)
                    
                break
            else:
#                 print("không trùng term nào cả, term:", term)
                ith_end_term -= 1
                term = GetTerm(sentence_part_tokenzied, ith_start_term, ith_end_term)

        # Đã xét xong term cuối cùng thì chuyển sang câu (vế) khác
        if ith_end_term == len(sentence_part_tokenzied):
            break
        # Nếu vẫn còn term để xét thì chuyển sang term kế tiếp
        else:
            # Đề phòng trường hợp term không có trong dictionary thì end sẽ tiến về và bằng start, và sẽ gây ra vòng lặp vô tận
            # nếu cứ gắn start bằng lại end (giá trị start ban đầu) và cộng end = start + LEN_MAX_TERM_VIETNAMESE
            # vd: term = "2 mũi vaccine cơ", start = 16, end = 20, chữ "2" không có trong dictionary nên sẽ khiến end bị trừ
            # thành 16 (bằng với start), và nếu không có điều kiện kiểm tra "ith_start_term == ith_end_term" thì sẽ khiến
            # start và end trở lại như cũ (16 và 20), vì ở đây gán start = end và đoạn bên dưới gán end = start + LEN_MAX_TERM_VIETNAMESE
            # Nên mới phải cần thêm term không có trong dictionary vào trong danh sách các term của từng văn bản (câu)
            # Và chuyển sang so khớp term kế tiếp
            if ith_start_term == ith_end_term:
                term = GetTerm(sentence_part_tokenzied, ith_start_term, ith_start_term+1)
                if ith_sentence not in sentences_tokenized.keys():
                    sentences_tokenized[ith_sentence] = [(term, ith_start_term, ith_end_term-1)]
                else:
                    sentences_tokenized[ith_sentence].append((term, ith_start_term, ith_end_term-1))
                
                ith_start_term = ith_end_term + 1
            else:
                ith_start_term = ith_end_term
                
            # Nếu vẫn còn nhiều hơn 4 term để xét thì chuyển sang 4 term kế tiếp
            if len(sentence_part_tokenzied) - ith_end_term > LENGTH_MAX_TERM:
                ith_end_term = ith_start_term + LENGTH_MAX_TERM
            # Nếu chỉ còn nhiều nhất là 4 term thì lấy ra những term cuối cùng của vế để xét
            else:
                ith_end_term = len(sentence_part_tokenzied)
                
            term = GetTerm(sentence_part_tokenzied, ith_start_term, ith_end_term)

In [19]:
for ith, sentence in enumerate(sentences):
    print(sentence)
    print(sentences_tokenized[ith], "\n")
# for key in  sentences_tokenized.keys():
#     print(sentences_tokenized[key])

Ngày 16/11, thông tin từ Ban chỉ đạo phòng chống dịch Covid-19 huyện Hải Hậu, Nam Định đã ghi nhận tới 52 ca Covid-19 cộng đồng.
[('ngày', 0, 0), ('16', 1, 0), ('11', 2, 1), ('thông tin', 3, 4), ('từ', 5, 5), ('ban', 6, 6), ('chỉ đạo', 7, 8), ('phòng chống', 9, 10), ('dịch', 11, 11), ('covid', 12, 11), ('19', 13, 12), ('huyện', 14, 14), ('hải', 15, 14), ('hậu', 16, 16), ('nam', 17, 17), ('định', 18, 18), ('đã', 19, 19), ('ghi nhận', 20, 21), ('tới', 22, 22), ('52', 23, 22), ('ca', 24, 24), ('covid', 25, 24), ('19', 26, 25), ('cộng đồng', 27, 28)] 

Đến nay, lực lượng chức năng đã xác định trên 600 F1 và hàng nghìn F2 liên quan nhóm bệnh nhân này.
[('đến nay', 0, 1), ('lực lượng', 2, 3), ('chức năng', 4, 5), ('đã', 6, 6), ('xác định', 7, 8), ('trên', 9, 9), ('600', 10, 9), ('f1', 11, 10), ('và', 12, 12), ('hàng', 13, 13), ('nghìn', 14, 14), ('f2', 15, 14), ('liên quan', 16, 17), ('nhóm', 18, 18), ('bệnh nhân', 19, 20), ('này', 21, 21)] 

Dự báo, số F0 tại xã Hải Minh dự kiến sẽ còn gia 